# Sending Logs using Python

## Import OpenTelemetry Modules for Logs

In [2]:
from opentelemetry.sdk.resources import Resource
from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import (
    OTLPLogExporter,
)
from opentelemetry.sdk._logs.export import (
    ConsoleLogExporter,
    BatchLogRecordProcessor,
    SimpleLogRecordProcessor
)
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
import logging, socket, sys, uuid;


## Create a **LoggerProvider** that we can send logs to

Create a [LoggerProvider](https://github.com/open-telemetry/opentelemetry-specification/blob/main/specification/logs/api.md#loggerprovider) with some **Resource**-level [Semantic Attributes](https://github.com/open-telemetry/semantic-conventions/blob/main/docs/resource/README.md) that describe our service:

In [3]:
provider = LoggerProvider(
    resource=Resource.create(
        {
            "service.name": __name__,
            "deployment.environment": "otel",
            "host.name": socket.gethostname(),
            "datadog.host.use_as_metadata": True,            
        }
    ),
)
print(provider._resource.__dict__)
# set_logger_provider(provider)

{'_attributes': {'telemetry.sdk.language': 'python', 'telemetry.sdk.name': 'opentelemetry', 'telemetry.sdk.version': '1.27.0', 'service.name': '__main__', 'deployment.environment': 'otel', 'host.name': 'COMP-M9FQMQQM7L', 'datadog.host.use_as_metadata': True}, '_schema_url': ''}


## Get a **Logger** from the **LoggerProvider**

Next we create a [SimpleLogRecordProcessor](https://github.com/open-telemetry/opentelemetry-python/blob/main/opentelemetry-sdk/src/opentelemetry/sdk/_logs/_internal/export/__init__.py#L113) that uses an [OTLPLogExporter](https://opentelemetry-python.readthedocs.io/en/latest/exporter/otlp/otlp.html#opentelemetry.exporter.otlp.proto.grpc._log_exporter.OTLPLogExporter) that is used to send the log event to our OTEL Collector.

> NOTE: Unlike traces that are comprised of multiple spans that can be sent independently and batched together with a **BatchSpanProcessor**, log events are single records and we can use the **SimpleLogRecordProcessor** to speed up the sending of the log record to Datadog. This sacrifices performance for quick throughput and is **not** recommended for production code.

In [4]:
log_exporter = OTLPLogExporter(endpoint="localhost:4317", insecure=True)
provider.add_log_record_processor(SimpleLogRecordProcessor(log_exporter))
# provider.add_log_record_processor(SimpleLogRecordProcessor(ConsoleLogExporter()))
# logger_provider.add_log_record_processor(BatchLogRecordProcessor(log_exporter))
# logger_provider.add_log_record_processor(BatchLogRecordProcessor(ConsoleLogExporter()))

## Attach an OTLP handler to the root logger

In [7]:
logger = logging.getLogger()
logger.handlers.clear()
logger.addHandler(logging.StreamHandler(sys.stderr))

# Attach OTLP handler to root logger
handler = LoggingHandler(logger_provider=provider)
logger.addHandler(handler)

## Create log events and send them to the collector

### Open Log Live Tail in Datadog

Open the live tail here: [Live Tail](https://app.datadoghq.com/logs/livetail?query=%40service.name%3A__main__)


## Send the Log Events

In [8]:
id=str(uuid.uuid4())
logging.debug(f"This is a DEBUG message, id:{id}")
logging.info(f"This is an INFO message, id:{id}")
logging.warning(f"This is a WARNING message, id:{id}")
logging.error(f"This is an ERROR message, id:{id}")

This is a WARNING message, id:f28f3a2b-ffc8-4409-9c57-21118bd3d308
This is an ERROR message, id:f28f3a2b-ffc8-4409-9c57-21118bd3d308


Notice the Collector emits a line indicating that it's processed a Log event:

```
2024-11-05T14:57:58.093-0600    info    Logs    {"kind": "exporter", "data_type": "logs", "name": "debug/basic", "resource logs": 2, "log records": 2}
```

and observe the Live Tail output in Datadog:

<img src="imgs/livetail-warnerr.png" width="600px"/>

<div class="alert alert-block alert-warning"><b>NOTE:</b> What's missing?</div>

In [16]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

id=str(uuid.uuid4())
logger.debug(f"This is a DEBUG message, id:{id}")
logger.info(f"This is an INFO message, id:{id}")
logger.warning(f"This is a WARNING message, id:{id}")
logger.error(f"This is an ERROR message, id:{id}")

This is a DEBUG message, id:76d63df3-49a9-4e19-83d9-58e20130afa8
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, ignoring call
Processor is already shutdown, igno

RecursionError: maximum recursion depth exceeded

## Shutdown the logger when complete

In [15]:
provider.shutdown()